# Calculate Optimal Tilt for Fixed-Axis Arrays
NOTE: Requires BigPanelTilt.yml environment 

This script uses _pvlib_ and lat, long, mount tech, and avgAzimuth to estimate the optimal tilt based on the maximizing Plane of Array (POA) irradiance at a given site and topography.


# Import Libraries and Variables

In [3]:
# Import libraries
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import pvlib
import warnings

In [4]:
# Set paths
wd = r'S:\Users\stidjaco\R_files\BigPanel'
derived_path = os.path.join(wd, r'Data/Derived')
derivedTemp_path = os.path.join(derived_path, r'intermediateProducts')

# Set final gmseusArrays path from script7 outputs
gmseusArraysInputPath = os.path.join(derivedTemp_path, r'GMSEUS_Arrays_wGEOID.shp')

# Set output path
gmseusArraysTiltPath = os.path.join(derivedTemp_path, r'GMSEUS_Arrays_estTilt.shp')

# Estimate Tilt
GM-SEUS Arrays (gmseusArrays) contains the following attributes for estimating tilt: 'avgAimuth' (degrees from north), 'GCR1' (rowArea / totArea), 'mount' (e.g., 'fixed_axis'), and 'latitude' and 'longitude'.\
We have also aready checked existing array infomation for tilt, so we will call estimated tilt a new column:'tiltEst'.\
Mounts with tilt information include: 'fixed_axis','mixed_fs', 'mixed_df', 'mixed_dfs', or 'mixed'.\
For each  row in the gmseusArrays DataFrame, find the angle of tilt that maximizes annual global POA irradiance for a local TMY and azimuth, using latitude and longitude of the location to get the TMY data and avgAzimuth as azimuth.\

In [11]:
# This cell requires 200+  minutes (~3.5 hours) to run. 

# Call gmseusArrays
gmseusArrays = gpd.read_file(gmseusArraysInputPath)

# Initialize the tiltEst column
gmseusArrays['tiltEst'] = int(-9999) # NOTE: Change to float if we decide to look beyond 1 degree increments

# Get gmseusArrays with mounts containing: 'fixed_axis','mixed_fs', 'mixed_df', 'mixed_dfs', or 'mixed'. Reset index.
gmseusArrays_tiltEst = gmseusArrays[gmseusArrays['mount'].str.contains('fixed_axis|mixed_fs|mixed_df|mixed_dfs|mixed', case=False)].reset_index(drop=True)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Estimate Optimum Tilt

# Suppress specific warnings globally (e.g., DeprecationWarning, FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Loop through the rows of the gmseusArrays DataFrame to perform the tilt estimation for each array
for index, row in gmseusArrays_tiltEst.iterrows():

    # Define the latitude and longitude of the  array location
    latitude = row['latitude']
    longitude = row['longitude']

    # Use the pvgis tool to get location specific TMY data
    # Global TMY data from pvgis
    tmy_data = pvlib.iotools.get_pvgis_tmy(latitude, longitude, outputformat='basic')
    
    # Get the weather data from TMY output (outputs: data, months_selected, inputs, metadata)
    tmy_data = tmy_data[0]
    
    # Ensure the index is a DatetimeIndex
    tmy_data.index = pd.to_datetime(tmy_data.index)

    # Define the location and use it to get solar position data that corresponds to the TMY data
    location = pvlib.location.Location(latitude, longitude)
    solar_position = location.get_solarposition(tmy_data.index)

    # Create a data frame for the results of panel tilet and total annual global POA irradiance
    results = pd.DataFrame(columns=['array_tilt', 'total_global_poa'])

    # The general rule of thumb is that the tilt of the array should be equal to the latitude of the location, with a 10 degree adjustment for the season. 
    # Latittude in the US ranges from 24 to 49 degrees, so to be concervative, we will loop through 10 to 70 degrees.
    # Loop through the possible array tilts from 0 to 90 degrees and calculate the total annual global POA irradiance
    for array_tilt in range(10, 70, 1): # by 1 degree means the output is an integer
        poa_irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt=array_tilt,
        surface_azimuth=row['avgAzimuth'],
        dni=tmy_data['dni'],
        ghi=tmy_data['ghi'],
        dhi=tmy_data['dhi'],
        solar_zenith=solar_position['zenith'],
        solar_azimuth=solar_position['azimuth'],)

        # Sum the Global POA Irradiance to get the total annual global POA irradiance
        total_global_poa = poa_irradiance['poa_global'].sum()
        
        # Add the results to the results DataFrame
        new_row = pd.DataFrame({'array_tilt': [array_tilt], 'total_global_poa': [total_global_poa]})
        results = pd.concat([results, new_row], ignore_index=True)
        
    # find the maximum total annual energy and the corresponding array tilt
    max_poa = results['total_global_poa'].max()
    est_tilt = results.loc[results['total_global_poa'] == max_poa, 'array_tilt'].values[0]

    # Assign the best tilt to the gmseusArrays DataFrame under the column tiltEst
    gmseusArrays_tiltEst.loc[index, 'tiltEst'] = int(est_tilt) # NOTE: Change to float if we decide to look beyond 1 degree increments

    # Print percent progress every 5% (round to nearest 5%)
    if (index + 1) % int(gmseusArrays_tiltEst.shape[0]/20) == 0:
        print('Progress:', round( (index+1) /gmseusArrays_tiltEst.shape[0]*100), '%')

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Prepare tilt and tiltEst for validation and export

# Drop gmseusArrays where arrayID is in gmseusArrays_tiltEst, reset index, and concatenate with gmseusArrays_tiltEst
gmseusArrays = gmseusArrays[~gmseusArrays['arrayID'].isin(gmseusArrays_tiltEst['arrayID'])]
gmseusArrays = gmseusArrays.reset_index(drop=True)
gmseusArrays = pd.concat([gmseusArrays, gmseusArrays_tiltEst], ignore_index=True)

# If mount string contains 'fixed','mixed_fs', 'mixed_df', 'mixed_dfs', or 'mixed', maintain tilt, otherwise set tilt to -9999. These come from checked arrays (errors in permitting data?) -- even for checked dat
gmseusArrays['tilt'] = gmseusArrays.apply(lambda row: row['tilt'] if any(x in str(row['mount']).lower() for x in ['fixed_axis', 'mixed_fs', 'mixed_df', 'mixed_dfs', 'mixed']) else -9999, axis=1)

# Ensure fill NaN in tilt and tiltEst with -9999
gmseusArrays['tilt'] = gmseusArrays['tilt'].fillna(-9999)
gmseusArrays['tiltEst'] = gmseusArrays['tiltEst'].fillna(-9999)

# ~~~~~~~~~~~~~~~~~~~~~~ Prepare validation export

# Select arrays with both tilt and tiltEst not equal to -9999
gmseusArrays_tiltValidate = gmseusArrays[(gmseusArrays['tilt'] != -9999) & (gmseusArrays['tiltEst'] != -9999)]

# Print number of arrays with both tilt and tiltEst not equal to -9999
print('Number of arrays with both tilt and tiltEst:', gmseusArrays_tiltValidate.shape[0])

# Export for validation in script8
gmseusArrays_tiltValidate.to_file(os.path.join(derivedTemp_path, r'GMSEUS_Arrays_wTilt_valid.shp'))

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Fill gaps and export 

# Where tilt is NaN (here, -9999), fill with tiltEst
gmseusArrays['tilt'] = gmseusArrays['tilt'].where(gmseusArrays['tilt'] != -9999, gmseusArrays['tiltEst'])

# Print number of arrays with tilt
print('Number of arrays with tilt:', gmseusArrays[gmseusArrays['tilt'] != -9999].shape[0])

# Export the final capacity dataset
gmseusArrays.to_file(gmseusArraysTiltPath)

Progress: 0 %
Progress: 4 %
Progress: 9 %
Progress: 14 %
Progress: 19 %
Progress: 24 %
Progress: 29 %
Progress: 34 %
Progress: 39 %
Progress: 44 %
Progress: 49 %
Progress: 54 %
Progress: 59 %
Progress: 64 %
Progress: 69 %
Progress: 74 %
Progress: 79 %
Progress: 84 %
Progress: 89 %
Progress: 94 %
Progress: 99 %
Number of arrays with both tilt and tiltEst: 1986
Number of arrays with tilt: 6689


In [12]:
# Perform Checks: Seperately, print the number of arrays where tilt is NaN, -9999, and not -9999 or NaN
print('Number of arrays with NaN tilt:', gmseusArrays[gmseusArrays['tilt'].isna()].shape[0])
print('Number of arrays with -9999 tilt:', gmseusArrays[gmseusArrays['tilt'] == -9999].shape[0])
print('Number of arrays with tilt:', gmseusArrays[gmseusArrays['tilt'] != -9999].shape[0])

Number of arrays with NaN tilt: 0
Number of arrays with -9999 tilt: 8328
Number of arrays with tilt: 6689
